In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [2]:
# Loading targets into stk from file.
stk_object.set_sim_time(days=1)
stk_object.Target_Loader("../../Input_Files/Targets_15.txt")
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

In [3]:
stk_object.Generate_Pre_Planning_Data()

- Computing_Access [==========] 180/180 [100%] in 0.5s (380.24/s)               
- Getting_AzEl [==========] 180/180 [100%] in 4.7s (38.60/s)                    
- Sorting_Data [==========] 4860/4860 [100%] in 0.0s (140273.39/s)              


0

In [4]:
stk_object.Plan(1,20)

- Planning [==========] 4860/4860 [100%] in 0.6s (11964.63/s)                   


0

In [5]:
data_comparison = {}
data_comparison["Unplanned (%)"] = [np.count_nonzero(stk_object.target_bins[tar_num])/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Unplanned (Time)"] = [np.max(stk_object.target_times[tar_num])/86400 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (%)"] = [len(np.unique(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (Time)"] = [max(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Time'].values)/86400 for tar_num in range(len(stk_object.targets))]

data_comparison = pd.DataFrame(data_comparison)
print(data_comparison)
print(data_comparison.describe())

    Unplanned (%)  Unplanned (Time)  Planned (%)  Planned (Time)
0       85.185185               1.0    53.703704        0.999843
1       75.925926               1.0    49.382716        0.937653
2       80.864198               1.0    37.345679        0.956267
3       84.567901               1.0    60.185185        0.954174
4       89.197531               1.0    51.851852        0.951234
5       73.148148               1.0    44.135802        0.999958
6       84.259259               1.0    41.666667        0.999404
7       88.580247               1.0    44.753086        0.938394
8       85.185185               1.0    43.209877        0.937921
9       92.901235               1.0    51.234568        0.941352
10      91.049383               1.0    49.074074        0.941768
11      95.987654               1.0    55.555556        0.942191
12      82.407407               1.0    50.000000        0.938153
13      94.444444               1.0    57.716049        0.941157
14      91.049383        